[주제] 테크 뉴스레터 작성하기

# CrewAI와 Tool 결합

In [47]:
import warnings
warnings.filterwarnings('ignore')

- Serper : Google Search API 키 발급
    - https://serper.dev/

In [48]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 라이브러리 임포트

### crewai_tools
- SerperDevTool : Serper api를 활용한 결과물 받는 도구, 구글 웹검색 도구
- WebsiteSearchTool : 특정 키워드와 특정 웹사이트를 주면, 해당 웹 사이트에서 키워드와 관련이 있는 부부만 가져옴.
- ScrapeWebsiteTool : 어떤 URL을 주면 해당 url에 있는 텍스트를 스크래핑해서 가져오는 도구

In [49]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# 필요한 객체 생성

In [50]:
# llm 객체
llm = ChatOpenAI(model="gpt-4o-mini")

# Tool
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

# Agent 정의

In [61]:
# 테크 트렌드 연구원
researcher = Agent(
    name="tech_trend_researcher",
    role = "테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2026년 1월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    verbose=True,
    max_iter=5,  # 해당 Agent가 주어진 task를 수행할 때 반복 실행할 횟수를 제한하는 것임(웹검색을 5번 실행, 비용과 연결되어 있음.)
    llm=llm
)

- allow_delegation=False
    - 해당 agnet가 task를 수행할 때 혼자 힘으로 하기 힘들 때 다른 agnet에 위임하는 설정
    - detault=True
    - False로 하지 않으면 수행중에 researcher에게 작업을 넘기는 경우가 발생하는데 이것을 발생하지 않도록 제한하기 위함

In [62]:
# 뉴스레터 작성 agnet
writer = Agent(
    name="tech_newsletter_writer",
    role="뉴스레터 작성자",
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2026년 1월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    verbose=True,
    allow_delegation=False,  
    llm=llm
)

# Task 정의

In [63]:
research = Task(
    name="tech_trend_research",
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

In [64]:
write = Task(
    name="tech_newsletter_writing",
    description="""
테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
""",
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터", # 결과물 형식
    agent=writer,
    output_file=r"./new_post.md" # the final blog post will be saved here
)

# Crew 정의

In [65]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=False,
    process=Process.sequential,
    output_log_file="logs/crew_run.json",   # 👈 JSON 형태로 모든 스텝이 저장됩니다.
)

# Crew 실행

In [66]:
import json

result = crew.kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 테크 트렌드 연구원                                                                                      │
│                                                                                                                 │
│  Task: AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': '2026 AI technology trends', 'type': 'search', 'num': 10, 'engine': 'google'},      │
│  'organic': [{'title': 'The trends that will shape AI and tech in 2026', 'link':                                │
│  'https://www.ibm.com/think/news/ai-tech-trends-predictions-2026', 'snippet': 'Reporter Anabelle Nicoud spoke   │
│  to several experts across AI, security, quantum and beyond to better understand where tech will take us in     │
│  2026.', 'position': 1}, {'title': "What's next in AI: 7 trends to watch in 2026", 'link':                      │
│  'https://news.microsoft.com/source/features/ai/whats-next-in-ai-7-trends-to-watch-in-2026/', 'snippet':        │
│  'Seven AI trends to watch in 2026 will make AI a true partner — boosting teamwork, security, research          │
│  momentum and infrastructure efficiency.', 'position': 2}, {'title': 'Five Trends in AI and Data Science for    │
│  2026', 'link': 'https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2026/', 'snippet':  │
│  "1. The AI bubble will deflate, and the economy will suffer. · 2. More all-in adopters will create 'AI         │
│  factories' and infrastructure. · 3. · 4.", 'position': 3}, {'title': 'Tech Trends 2026 | Deloitte Insights',   │
│  'link': 'https://www.deloitte.com/us/en/insights/topics/technology-management/tech-trends.html', 'snippet':    │
│  'AI goes physical: Navigating the convergence of AI and robotics · The agentic reality check: Preparing for a  │
│  silicon-based workforce · The AI ...', 'position': 4}, {'title': "What's next for AI in 2026 | MIT Technology  │
│  ...", 'link': 'https://www.technologyreview.com/2026/01/05/1130662/whats-next-for-ai-in-2026/', 'snippet':     │
│  'In 2026, expect more Silicon Valley apps to quietly ship on top of Chinese open models, and look for the lag  │
│  between Chinese releases and the ...', 'position': 5}, {'title': 'Five Significant Tech Trends That Will       │
│  Feature in 2026', 'link':                                                                                      │
│  'https://internationalbanker.com/technology/five-significant-tech-trends-that-will-feature-in-2026/',          │
│  'snippet': 'In contrast, 2026 may well be the year of agentic AI. This trend is underpinned...                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  The trends that will shape AI and tech in 2026 | IBM                                                           │
│                                                                                                                 │
│  Artificial Intelligence                                                                                        │
│                                                                                                                 │
│  Statements and reactions                                                                                       │
│                                                                                                                 │
│  The trends that will shape AI and tech in 2026                                                                 │
│                                                                                                                 │
│  Published 01 January 2026                                                                                      │
│                                                                                                                 │
│  Anabelle Nicoud                                                                                                │
│                                                                                                                 │
│  Staff Writer                                                                                                   │
│                                                                                                                 │
│  IBM                                                                                                            │
│                                                                                                                 │
│  This article was featured in the Think newsletter. Get it in your inbox .                                      │
│                                                                                                                 │
│  A year in tech can feel like a decade anywhere else.                                                           │
│                                                                                                                 │
│  Think about it: a year ago, we were discussing how ChatGPT wasn’t able to count the number of “r”s in          │
│  “strawberry.” Reasoning models from Chinese frontier labs (like DeepSeek-R1 ) hadn’t taken the world by        │
│  storm, and neither had open-source reasoning agents.                                                           │
│                                                                                                                 │
│  Claude’s dedicated coding agent didn’t exist yet. IBM’s Granite 3.0 had only just arrived . And the agent      │
│  conversation was only beginning: MCP had just gained traction in the spring, with a notable endorsement from   │
│  Sam Altman.                                                                                                    │
│                                                                                                                 │
│  Meanwhile, in the world of infrastructure, chips and compute resources were becoming scarce , giving new       │
│  territories a competitive advantage.                                                                           │
│                                                                                                                 │
│  Over the last few weeks, IBM Think spoke with a dozen

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 테크 트렌드 연구원                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **2026년 AI 업계의 최신 기술 동향**                                                                            │
│                                                                                                                 │
│  1. **AI와 물리적 로봇의 융합**: 2026년은 AI 기술과 로봇 기술의 융합이 가속화되는 해로 예상됩니다. AI가 단순한  │
│  질문 답변을 넘어 실제 물리적 환경에서 감지하고 행동하며 학습할 수 있는 시스템으로 발전할 것입니다. 이는        │
│  복잡한 작업을 완전하게 자동화하는 능력으로 이어져, 기업은 자신의 비즈니스 프로세스를 효율적으로 개선할 수      │
│  있는 새로운 가능성을 열게 됩니다. 피터 스타르(Peter Staar), IBM 연구소의 수석 연구원이 이끄는 예측에 따르면,   │
│  대형 언어 모델의 지배가 여전히 이어지지만, 산업 전반에서 새로운 아이디어에 대한 갈망이 커지고 있으며, 이는     │
│  물리적 AI의 전환으로 이어질 것입니다.                                                                          │
│                                                                                                                 │
│  2. **AI의 소형화 및 에지 컴퓨팅의 발전**: 마이크로 모델과 도메인 최적화 모델이 점점 더 중요해질 것입니다.      │
│  이는 에지 장치에서의 AI 연산을 가능하게 하여, 비용 및 데이터 주권 요구에 따라 로컬 환경에서 AI를 직접 구현할   │
│  수 있게 합니다. 맷 화이트(Matt White), 파이토치 재단의 경영 이사는 작은 모델의 중요성을 강조하며, 향후 전      │
│  세계적으로 모델의 다양성이 증가할 것이라고 예측합니다. AI 프로젝트의 중단을 피하기 위해 기업들은 AI 주권을     │
│  확보하는 것이 필수적이며, 많은 경영진들이 이를 전략에서 고려해야 할 본질적 요소로 인식하고 있습니다.           │
│                                                                                                                 │
│  3. **개방형 AI의 발전과 관리**: 파이토치의 개방형 AI 플랫폼은 개발자들에게 멀티모달 추론 및 안전성을 강화한    │
│  평가를 위한 유연한 도구를 제공할 것입니다. AI의 개방성은 문제 해결 속도를 높이고 다양한 환경에서의 협업을      │
│  촉진할 것입니다. 이는 궁극적으로 AI의 투명성과 신뢰성을 높이는데 기여할 것입니다. 앤서니 마셜(Anthony          │
│  Marshall), IBM의 비즈니스 가치 연구소의 수석 이사는 AI의 투명한 운영을 요구하는 규제와 소비자의 요구가 증가할  │
│  것이라고 강조하고 있습니다.                                                                                    │
│                                                                                                                 │
│  AI 업계는 2026년에 더욱 진화하여 새로운 비즈니스 모델을 창출하고, 기술적 한계를 뛰어넘어 더 안전하고 신뢰할    │
│  수 있는 환경을 조성하는 방향으로 나아갈 것입니다. 이러한 변화는 기업이 AI를 활용하여 더 많은 가치를 창출하는   │
│  데 기여할 것으로 예상됩니다.                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 뉴스레터 작성자                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│  테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.                       │
│  테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 뉴스레터 작성자                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 2026년 AI 테크 뉴스레터: 로봇의 진화와 소형화의 시대                                                         │
│                                                                                                                 │
│  안녕하세요, AI 기술의 매력을 사랑하는 여러분! 오늘은 2026년을 맞아 AI 산업에서 사람들이 기대하는 흥미로운      │
│  혁신들을 소개하려고 해요. AI와 로봇 기술의 융합이 가속화되면서, 이제는 물리적 공간에서도 AI가 실질적인 행동을  │
│  할 수 있는 날이 오고 있습니다. IBM의 스타 연구원 피터 스타르에 따르면, AI는 단순한 질문 답변을 넘어 복잡한     │
│  업무를 자동화해 기업의 비즈니스 프로세스를 혁신적으로 변화시킬 것으로 기대됩니다. 상상해보세요, 자동으로 물류  │
│  관리가 이루어지는 현장을! 일의 효율성이 크게 향상될 거예요.                                                    │
│                                                                                                                 │
│  그리고 또 하나 주목할 점은 AI의 소형화와 에지 컴퓨팅 발전이에요. 마이크로 모델과 도메인 최적화 모델의          │
│  중요성이 갈수록 커지고 있습니다. 파이토치 재단의 맷 화이트 이사가 말했듯이, 이제는 로컬 환경에서 AI를 직접     │
│  구현할 수 있는 시대가 다가오고 있습니다. 기업들은 자신들의 데이터 주권을 확보하기 위해 AI를 소형화하는 전략을  │
│  시급히 고려해야 할 것입니다. 이렇게 되면 비용 절감과 성능 향상을 노릴 수 있겠죠!                               │
│                                                                                                                 │
│  또한, 개방형 AI의 발전도 눈여겨볼 요소입니다. 파이토치의 개방형 AI 플랫폼은 멀티모달 추론을 지원하며, 다양한   │
│  환경에서의 협업을 유도해 문제 해결 속도를 높일 것입니다. IBM의 앤서니 마셜 이사가 언급했듯이, 투명한 AI        │
│  운영에 대한 수요가 증가하면서, 개발자들은 투명성을 증대시키기 위한 도구와 방법들을 더욱 정교하게 만들어        │
│  나가야 합니다. 결과적으로 AI의 신뢰성이 강해지고, 다양한 사람들과 협력하는 기반이 마련될 것입니다.             │
│                                                                                                                 │
│  2026년 AI 업계는 더욱 진화하며 새로운 비즈니스 모델을 창출하고, 기술적 한계를 뛰어넘는 혁신을 이룰 것입니다.   │
│  기업들이 AI를 통해 더욱 큰 가치를 창출하는 요인이 이 남다른 변화에 있다고 할 수 있죠. 지금이 바로 우리가 AI의  │
│  미래를 함께 탐험할 최고의 순간입니다! 다음 호에서 또 만나요!                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# 마크다운으로 표시

In [45]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# 최신 AI 기술 트렌드 뉴스레터 (2026년 1월)

안녕하세요, 테크 애호가 여러분! 2026년이 밝았습니다. 새로운 해, 새로운 시작! 오늘은 인공지능(AI) 분야에서 가장 주목할 만한 기술 트렌드를 살펴보겠습니다. AI가 세상을 어떻게 변화시키고 있는지, 그리고 이것이 우리 삶에 어떤 영향을 미치는지를 아는 것은 불가결한 요소가 되었죠. 그럼, 주요 트렌드를 한 번 알아볼까요?

## Generative AI의 혁신

최근 몇 년간 우리는 Generative AI의 눈부신 발전을 목격하였습니다. 이 기술은 이제 단순한 호기심의 대상이 아니라, 텍스트, 이미지, 음악 등 다양한 콘텐츠를 창작하는 데에 필수적인 도구로 자리 잡았습니다. OpenAI의 GPT 모델과 DALL-E와 Midjourney와 같은 이미지 생성 모델들은 창작자들에게 무한한 아이디어의 원천을 제공하고 있답니다. 이러한 변화는 콘텐츠 제작의 패러다임을 바꿔놓고 있으며, 개인 맞춤형 서비스를 통해 조금 더 진화된 비즈니스 모델을 탐험할 수 있는 기회를 제공하고 있습니다.

## 윤리적 규범과 규제의 필요성

AI의 급속한 발전에 따라 윤리적 사회적 문제도 불거지고 있습니다. 개인 정보 보호, 저작권, 알고리즘의 편향성 등 여러 이슈들이 대두되고 있죠. 올해는 정부와 기업이 협력하여 이 문제를 적극적으로 해결하려는 움직임이 강화될 것으로 보입니다. AI 개발자들에게는 윤리적 기준을 준수하는 것만큼 중요한 과제가 없겠죠. 이는 기업의 신뢰도를 높여 소비자들로부터 긍정적인 반응을 불러올 준비가 되어 있다는 신호입니다!

## AI와 엣지 컴퓨팅의 융합

사물인터넷(IoT) 기기의 확산과 함께, 엣지 컴퓨팅 기술이 더욱 중요해지고 있습니다. 이제 AI는 데이터 전송의 지연을 최소화하면서 엣지 디바이스에서 직접 처리할 수 있게 되었답니다. 이러한 혁신은 산업 자동화, 자율주행차 및 스마트 홈 기술 등에서 운영 효율성을 극대화하고, 비용 절감의 길을 열어줍니다. 뿐만 아니라, 엣지 AI는 데이터 보안을 강화하고 네트워크 대역폭의 효율성을 높이며, 데이터 처리 시간 역시 대폭 단축시키는데 기여할 것으로 보입니다.

이렇게 2026년, AI는 우리 생활의 모든 면에서 변화를 주도하고 있습니다. 이러한 기술 흐름을 정확히 이해하고 잘 활용하는 것이 회사의 생존 전략이 될 것입니다. AI가 가져올 미래의 변화를 기대하시고, 매력적인 기술의 세계에 함께 발을 담가보시길 바랍니다!

[tip] log 후처리
- logs/crew_run.json의 로그 결과에서 한글을 하기 위한 JSON 변환하기

In [46]:
import json

with open("logs/crew_run.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("logs/crew_run_readable.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("crew_run_readable.json 생성 완료")


crew_run_readable.json 생성 완료
